In [1]:
from utils import *
from nearest_neighbor import *
import matplotlib.pyplot as plt

In [2]:
path_data = "/Users/lou/Thesis/BCR_Clone_Identification/Data/"
path_sample = "/Users/lou/Thesis/BCR_Clone_Identification/Data/sample90/"

In [3]:
outfile = path_data+"Nt_info.csv"

In [4]:
df = pd.read_csv(outfile,sep='\t')
df

,Sequence number,Sequence ID,V-GENE,J-GENE,V-D-J-REGION,JUNCTION,JUNCTION length
0,1,M06023:233:000000000-JCVJP:1:1102:22274:8511_1...,Homsap IGHV5-51,Homsap IGHJ6,aagggttctggatacagttttaccaactactggatcggctgggtgc...,tgtgcgaggtcaaattatcatgatagtagtggttaccactactacg...,63.0
1,3,M06023:233:000000000-JCVJP:1:1102:23002:8543_1...,Homsap IGHV2-5,Homsap IGHJ4,acaccgaccctcacgctgacctgtaccttctctgggttctcactca...,tgtgcacacagctctgtcggggtcgacgggctggtccaggactttg...,54.0
2,4,M06023:233:000000000-JCVJP:1:1102:11694:8553_1...,Homsap IGHV2-5,Homsap IGHJ4,acacagaccctcacgctgacctgcaccttctctgggttctcactca...,tgtgcacacagctctgtaggggtcgacgggctggtccaggactttg...,54.0
3,5,M06023:233:000000000-JCVJP:1:1102:14232:8560_1...,Homsap IGHV1-46,Homsap IGHJ4,gcatctggatacatcttcaccagctactttatgcactgggtgcgac...,tgtgcgagagatgggtcgattgagtggctacgattgggctactgg,45.0
4,7,M06023:233:000000000-JCVJP:1:1102:9846:8586_1:...,Homsap IGHV2-5,Homsap IGHJ4,acacagaccctcacgctgacctgcaccgtctctgggttctcactca...,tgtgcacacagctctgtaggggtcgacgggctggtccaggactttg...,54.0
...,...,...,...,...,...,...,...
19190,81302,M06023:233:000000000-JCVJP:1:2119:18177:24811_...,Homsap IGHV1-46,Homsap IGHJ6,gcatctggatacaccttcaccagctactatatacactggctacgac...,tgtgcgaggtcaaattatcataataatagtggttaccactactacg...,63.0
19191,81306,M06023:233:000000000-JCVJP:1:2119:19235:24852_...,Homsap IGHV1-2,Homsap IGHJ4,gcttctggatacaccttcaccgactactatatgcactgggtgcgac...,tgtgcgagagatgggtcgattgtgtggccacgattgggctactgg,45.0
19192,81310,M06023:233:000000000-JCVJP:1:2119:17091:24945_...,Homsap IGHV1-2,Homsap IGHJ4,ggcctcagttctcctgcaaggcttctggatacaccttcaccgacta...,tgtgcgagagcctcagactactgg,24.0
19193,81312,M06023:233:000000000-JCVJP:1:2119:15970:24974_...,Homsap IGHV1-46,Homsap IGHJ4,gaatctggatacatcttcaccagctactatatgcactgggtgcgac...,tgtgcgagagatgggtcgattgagtggccacgattgggctactgg,45.0


## compute tf-idf representation

In [ ]:
%%time
%pdb
seqs_tf_idf = cal_tf_idf(df.loc[:,"V-D-J-REGION"].values,k=2,atoms=["a","t","c","g","n"])

Automatic pdb calling has been turned ON


KeyError: 'gn'

> /Users/lou/Thesis/BCR_Clone_Identification/Scripts/utils.py(160)bag_of_words()
    158     for i in range(len(sequence)-k+1):
    159         kmer = sequence[i:i+k]
--> 160         bag[kmer] += 1
    161     return bag
    162 

ipdb> kmer
'gn'
ipdb> sequence
'cactgtctctggtggctccatcagcagtgntgtttactactggagctggatccgccagcacccagggaagggcctggagtggattgggtacatctattacagtgggagcacctactacaacccgtccctcaagagtcgagttaccatatcagtagacacgtctaagaaccagttctccctgaagctgagctctgtgactgccgcggacacggccgtgtattactgtgcgagagggcgtggctacgattttacccttgactactggggccagggaaccct'


In [ ]:
%%time
d_to_nearest_all = dist_to_nearest_all_exhaustive(seqs_tf_idf,distance=Cosine_dist)